<a href="https://colab.research.google.com/github/wanruu/CS5487/blob/main/logistic_regression.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import torch 
import torch.nn as nn
from torch.autograd import Variable
# torchvision可以帮助我们处理常用数据集，如MNIST，COCO, ImageNET等
import torchvision.datasets as dsets
import torchvision.transforms as transforms
import numpy as np
import matplotlib.pyplot as plt
import torchvision
from skimage import data,img_as_float
# hyper parameter
input_size = 28 * 28 # image size of MNIST data
num_classes = 10
num_epochs = 50
batch_size = 64
learning_rate = 1e-3



In [ ]:
import scipy.io
from torch.utils.data import Dataset


class MyDataset(Dataset):
    def __init__(self, filepath, filetype="mat", train=True, cross_idx=0):
        # Read data from mat file.
        # Extract all data.
        if filetype == "mat":
            mat_data = scipy.io.loadmat(filepath)
            labels = mat_data["digits_labels"]  # (1,4000)
            imgs = mat_data["digits_vec"]  # (784,4000)
            trainset = mat_data["trainset"][cross_idx] - 1  # (2,2000) -> (2000,)
            testset = mat_data["testset"][cross_idx] - 1  # (2,2000) -> (2000,)
        if train:
            self.imgs = imgs[:,trainset]  # (784,2000)
            self.labels = labels[:,trainset]  # (1,2000)
        else:
            self.imgs = imgs[:,testset]  # (784,2000)
            self.labels = labels[:,testset] # (1,2000)

    def __getitem__(self, index):
        return self.imgs[:,index], self.labels[:,index]
        

    def __len__(self):
        return self.labels.shape[1]


# # Sample
# if __name__ == "__main__":
#     from config import DIGITS_MAT_PATH
#     train_cross_1 = MyDataset(DIGITS_MAT_PATH, "mat", True, 0)
#     test_cross_1 = MyDataset(DIGITS_MAT_PATH, "mat", False, 0)
#     train_cross_2 = MyDataset(DIGITS_MAT_PATH, "mat", True, 1)
#     test_cross_2 = MyDataset(DIGITS_MAT_PATH, "mat", False, 1)

In [ ]:
DIGITS_MAT_PATH='/content/drive/MyDrive/Colab_Notebooks/digits4000.mat'

train_cross_1 = MyDataset(DIGITS_MAT_PATH, "mat", True, 0)
test_cross_1 = MyDataset(DIGITS_MAT_PATH, "mat", False, 0)
train_cross_2 = MyDataset(DIGITS_MAT_PATH, "mat", True, 1)
test_cross_2 = MyDataset(DIGITS_MAT_PATH, "mat", False, 1)

In [ ]:
#model

#加载数据

train_loader = torch.utils.data.DataLoader(dataset = train_cross_1, 
                                           batch_size = batch_size, 
                                           shuffle = True)  
test_loader = torch.utils.data.DataLoader(dataset = test_cross_1,
                                          batch_size = batch_size,
                                          shuffle = True)

# 定义网络
class logistic_regression(nn.Module):
    def __init__(self,input_size, num_classes):
        super(logistic_regression, self).__init__()
        self.linear = nn.Linear(input_size, num_classes)

    def forward(self, x):
        out = self.linear(x)
        return out

model = logistic_regression(input_size, num_classes)
print(model)

logistic_regression(
  (linear): Linear(in_features=784, out_features=10, bias=True)
)


In [ ]:
for i, (images, labels) in enumerate(train_loader): #利用enumerate取出一个可迭代对象的内容
        images = Variable(images.view(-1, 28 * 28))
        labels = Variable(labels)
        break

In [ ]:
labels.shape

torch.Size([64, 1])

In [ ]:
labels.squeeze().shape

torch.Size([64])

In [ ]:
#train
# optimization

criterion = nn.CrossEntropyLoss() # 交叉熵损失
optimizer = torch.optim.SGD(model.parameters(), lr = learning_rate)

#print(train_loader)
model.train()
for epoch in range(100):
#for epoch in range(2):
    print(' epoch = %d' % epoch)
    for i, (images, labels) in enumerate(train_loader): #利用enumerate取出一个可迭代对象的内容
        images = Variable(images.view(-1, 28 * 28))
        labels = Variable(labels.squeeze())

        optimizer.zero_grad()
        outputs = model(images.float())
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        if  i % 100 == 0:
            print('current loss = %.5f' % loss.data)

    #print(np.shape(images))


 epoch = 0
current loss = 73.94708
 epoch = 1
current loss = 26.06924
 epoch = 2
current loss = 101.36839
 epoch = 3
current loss = 8.64742
 epoch = 4
current loss = 24.69844
 epoch = 5
current loss = 21.14187
 epoch = 6
current loss = 22.67946
 epoch = 7
current loss = 31.66484
 epoch = 8
current loss = 174.71194
 epoch = 9
current loss = 7.04019
 epoch = 10
current loss = 18.54757
 epoch = 11
current loss = 13.65016
 epoch = 12
current loss = 11.77247
 epoch = 13
current loss = 14.65736
 epoch = 14
current loss = 2.51129
 epoch = 15
current loss = 1.95141
 epoch = 16
current loss = 9.66569
 epoch = 17
current loss = 25.39056
 epoch = 18
current loss = 0.96778
 epoch = 19
current loss = 7.08971
 epoch = 20
current loss = 7.05995
 epoch = 21
current loss = 1.35463
 epoch = 22
current loss = 6.29378
 epoch = 23
current loss = 0.29007
 epoch = 24
current loss = 0.91907
 epoch = 25
current loss = 100.23642
 epoch = 26
current loss = 1.84908
 epoch = 27
current loss = 3.16808
 epoch = 28
c

In [ ]:
#test
# test the model
correct = 0
total = 0
model.eval()
for images, labels in test_loader:
    images = Variable(images.view(-1, 28 * 28))
    labels = Variable(labels.squeeze())
    outputs = model(images.float())
    #torch.max(x, n) 沿着n维进行某种操作。得到的是某一维度的最大值之类的，如果不加维度n，则返回所有元素的最大值之类的
    _, predicted = torch.max(outputs.data, 1)
    total += labels.size(0)
    print(total)
    print(correct)
    correct += (predicted == labels).sum()

print('accuracy of the model %.2f' % (100 * correct / total))

64
0
128
tensor(50)
192
tensor(108)
256
tensor(167)
320
tensor(227)
384
tensor(284)
448
tensor(339)
512
tensor(394)
576
tensor(448)
640
tensor(506)
704
tensor(556)
768
tensor(610)
832
tensor(667)
896
tensor(726)
960
tensor(784)
1024
tensor(837)
1088
tensor(893)
1152
tensor(948)
1216
tensor(1007)
1280
tensor(1066)
1344
tensor(1119)
1408
tensor(1177)
1472
tensor(1235)
1536
tensor(1286)
1600
tensor(1342)
1664
tensor(1395)
1728
tensor(1447)
1792
tensor(1505)
1856
tensor(1561)
1920
tensor(1613)
1984
tensor(1671)
2000
tensor(1722)
accuracy of the model 86.75


In [ ]:
predicted

tensor([9, 0, 7, 5, 2, 2, 6, 3, 3, 5, 0, 6, 3, 3, 2, 3])

In [ ]:
labels

tensor([[9],
        [0],
        [7],
        [5],
        [2],
        [5],
        [6],
        [8],
        [5],
        [3],
        [0],
        [6],
        [3],
        [3],
        [2],
        [3]], dtype=torch.uint8)